# 481n Project Advanced Solution

In [2]:
import cayde
from cayde.well.nlpwell import NLPWell
from cayde.classifier.dtree import DecisionTreeClassifier
from cayde.classifier.ada import AdaBoosterClassifier
from cayde.classifier.svm import SVMClassifier
from bert_tools.model import (
    create_model, run_config, 
    BATCH_SIZE, LEARNING_RATE, 
    NUM_TRAIN_EPOCHS, 
    WARMUP_PROPORTION, 
    SAVE_SUMMARY_STEPS,
    model_fn_builder
)

In [3]:
well = NLPWell("training_data.csv")
well.load()

well.input_cols = well.text_cols = ["head", "body"]
well.output_col = "Stance"

In [4]:
well.input_df

,head,body
2295,WFB’s Bill Gertz: Obama Administration ‘Trying...,Eleven planes missing from Tripoli Airport aft...
864,Nicaragua Meteorite Impact Theory May Be Meteo...,Nicaraguan officials say a meteorite gouged ou...
2905,Rapper Big Bank Hank of Sugarhill Gang dies at 57,"(CNN) -- A member of the Sugarhill Gang, whose..."
2800,ISIS fighters seize weapons airdrop meant for ...,The Pentagon admitted on Wednesday that ISIS d...
1470,Teacher sex tape exposed as a FAKE after inter...,Lucita Sandoval was not involved in a sex tape...
...,...,...
13,Indian civil servant sacked 'after 24-year sic...,An Indian public official has been sacked for ...
2196,Meteor Leaves 40-Foot Crater Near Managua's Ai...,Updated at 11:45 p.m. ET\r\n\r\nThere was an u...
532,An Indian Civil Servant Just Got Sacked After ...,"(Reuters) - Even in India, where government jo..."
2589,Rapper Big Bank Hank of Sugarhill Gang dies at 57,The music world is mourning the death of rappe...


In [4]:
# Work with a smaller dataset in a non-google TPU environment
well = well.getToySample(maxRows=1000)

In [5]:
well.clean_text(package='bert')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


['head_1gram', 'head_clean', 'body_1gram', 'body_clean']

In [5]:
well.save()
well._df

,index,Stance,head,body,head_1gram,head_clean,body_1gram,body_clean,head_body_bert
2295,2295,1,WFB’s Bill Gertz: Obama Administration ‘Trying...,Eleven planes missing from Tripoli Airport aft...,"[w, ##fb, ’, s, bill, ge, ##rt, ##z, :, obama,...",w ##fb ’ s bill ge ##rt ##z : obama administra...,"[eleven, planes, missing, from, tripoli, airpo...",eleven planes missing from tripoli airport aft...,<bert.run_classifier.InputFeatures object at 0...
864,864,1,Nicaragua Meteorite Impact Theory May Be Meteo...,Nicaraguan officials say a meteorite gouged ou...,"[nicaragua, meteor, ##ite, impact, theory, may...",nicaragua meteor ##ite impact theory may be me...,"[nicaragua, ##n, officials, say, a, meteor, ##...",nicaragua ##n officials say a meteor ##ite go ...,<bert.run_classifier.InputFeatures object at 0...
2905,2905,1,Rapper Big Bank Hank of Sugarhill Gang dies at 57,"(CNN) -- A member of the Sugarhill Gang, whose...","[rapper, big, bank, hank, of, sugar, ##hill, g...",rapper big bank hank of sugar ##hill gang dies...,"[(, cnn, ), -, -, a, member, of, the, sugar, #...",( cnn ) - - a member of the sugar ##hill gang ...,<bert.run_classifier.InputFeatures object at 0...
2800,2800,1,ISIS fighters seize weapons airdrop meant for ...,The Pentagon admitted on Wednesday that ISIS d...,"[isis, fighters, seize, weapons, air, ##drop, ...",isis fighters seize weapons air ##drop meant f...,"[the, pentagon, admitted, on, wednesday, that,...",the pentagon admitted on wednesday that isis d...,<bert.run_classifier.InputFeatures object at 0...
1470,1470,1,Teacher sex tape exposed as a FAKE after inter...,Lucita Sandoval was not involved in a sex tape...,"[teacher, sex, tape, exposed, as, a, fake, aft...",teacher sex tape exposed as a fake after inter...,"[luc, ##ita, sand, ##ova, ##l, was, not, invol...",luc ##ita sand ##ova ##l was not involved in a...,<bert.run_classifier.InputFeatures object at 0...
...,...,...,...,...,...,...,...,...,...
13,13,1,Indian civil servant sacked 'after 24-year sic...,An Indian public official has been sacked for ...,"[indian, civil, servant, sacked, ', after, 24,...",indian civil servant sacked ' after 24 - year ...,"[an, indian, public, official, has, been, sack...",an indian public official has been sacked for ...,<bert.run_classifier.InputFeatures object at 0...
2196,2196,1,Meteor Leaves 40-Foot Crater Near Managua's Ai...,Updated at 11:45 p.m. ET\r\n\r\nThere was an u...,"[meteor, leaves, 40, -, foot, crater, near, ma...",meteor leaves 40 - foot crater near mana ##gua...,"[updated, at, 11, :, 45, p, ., m, ., et, there...",updated at 11 : 45 p . m . et there was an une...,<bert.run_classifier.InputFeatures object at 0...
532,532,1,An Indian Civil Servant Just Got Sacked After ...,"(Reuters) - Even in India, where government jo...","[an, indian, civil, servant, just, got, sacked...",an indian civil servant just got sacked after ...,"[(, reuters, ), -, even, in, india, ,, where, ...","( reuters ) - even in india , where government...",<bert.run_classifier.InputFeatures object at 0...
2589,2589,1,Rapper Big Bank Hank of Sugarhill Gang dies at 57,The music world is mourning the death of rappe...,"[rapper, big, bank, hank, of, sugar, ##hill, g...",rapper big bank hank of sugar ##hill gang dies...,"[the, music, world, is, mourning, the, death, ...",the music world is mourning the death of rappe...,<bert.run_classifier.InputFeatures object at 0...


# BERT Experimentation

In [6]:
well.createBERTEncodings(compareTextMode=True)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 1000


INFO:tensorflow:Writing example 0 of 1000


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:label: 1 (id = 0)


INFO:tensorflow:label: 1 (id = 0)


['head_body_bert']

In [8]:
well._df['head_body_bert']

2295    <bert.run_classifier.InputFeatures object at 0...
864     <bert.run_classifier.InputFeatures object at 0...
2905    <bert.run_classifier.InputFeatures object at 0...
2800    <bert.run_classifier.InputFeatures object at 0...
1470    <bert.run_classifier.InputFeatures object at 0...
                              ...                        
13      <bert.run_classifier.InputFeatures object at 0...
2196    <bert.run_classifier.InputFeatures object at 0...
532     <bert.run_classifier.InputFeatures object at 0...
2589    <bert.run_classifier.InputFeatures object at 0...
686     <bert.run_classifier.InputFeatures object at 0...
Name: head_body_bert, Length: 1000, dtype: object

In [9]:
# Now we actually _run_ the BERT model : o
import tensorflow_hub as hub
import tensorflow as tf
from cayde.well.nlpwell import BERT_MODEL_HUB
import bert
from datetime import datetime

MAX_SEQ_LENGTH = 128

In [10]:
well.input_cols = ['head_body_bert']

In [11]:
well.df

,Stance,head_body_bert
2295,1,<bert.run_classifier.InputFeatures object at 0...
864,1,<bert.run_classifier.InputFeatures object at 0...
2905,1,<bert.run_classifier.InputFeatures object at 0...
2800,1,<bert.run_classifier.InputFeatures object at 0...
1470,1,<bert.run_classifier.InputFeatures object at 0...
...,...,...
13,1,<bert.run_classifier.InputFeatures object at 0...
2196,1,<bert.run_classifier.InputFeatures object at 0...
532,1,<bert.run_classifier.InputFeatures object at 0...
2589,1,<bert.run_classifier.InputFeatures object at 0...


In [12]:
# Compute # train and warmup steps from batch size
training_data = well.df
label_list = well.output_df[well.output_col].unique()
train_features = well.df['head_body_bert']
num_train_steps = int(len(training_data) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [16]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': '.cayde/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff03de80a90>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '.cayde/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff03de80a90>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [17]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [ ]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Saving checkpoints for 0 into .cayde/model.ckpt.


INFO:tensorflow:Saving checkpoints for 0 into .cayde/model.ckpt.


# API Demonstration

In [ ]:
# Returns a list of new columns available for usage
# This creates columns where the text is stemmed and cleaned
well.clean_text()

In [ ]:
well.input_cols = ['head', 'body', 'head_clean', 'body_1gram']

In [ ]:
well.df.head()

In [ ]:
# This creates columns where the text is converted to bigrams/trigrams
well.createNgrams()

In [ ]:
# Does the word exist in each text_column?
well.createCountWordFeatures([
    'fake',
    'fraud',
    'hoax',
    'false',
    'deny', 'denies',
    'refute',
    'not',
    'despite',
    'nope',
    'doubt', 'doubts',
    'bogus',
    'debunk',
    'pranks',
    'retract'
])

In [ ]:
well.input_cols = production_cols = [
 'head_2gram_unique_ratio',
 'head_3gram_unique_ratio',
 'head_4gram_unique_ratio',
 'body_2gram_unique_ratio',
 'body_3gram_unique_ratio',
 'body_4gram_unique_ratio'
] + ['head_contains_fake',
 'head_contains_fraud',
 'head_contains_hoax',
 'head_contains_false',
 'head_contains_deny',
 'head_contains_denies',
 'head_contains_refute',
 'head_contains_not',
 'head_contains_despite',
 'head_contains_nope',
 'head_contains_doubt',
 'head_contains_doubts',
 'head_contains_bogus',
 'head_contains_debunk',
 'head_contains_pranks',
 'head_contains_retract',
 'body_contains_fake',
 'body_contains_fraud',
 'body_contains_hoax',
 'body_contains_false',
 'body_contains_deny',
 'body_contains_denies',
 'body_contains_refute',
 'body_contains_not',
 'body_contains_despite',
 'body_contains_nope',
 'body_contains_doubt',
 'body_contains_doubts',
 'body_contains_bogus',
 'body_contains_debunk',
 'body_contains_pranks',
 'body_contains_retract'
]

In [ ]:
well.y

In [ ]:
clf = DecisionTreeClassifier(well)
clf.fit()
clf.accuracy()

In [ ]:
clf = AdaBoosterClassifier(well, n_estimators=150)
clf.fit()
clf.accuracy()

In [ ]:
clf = SVMClassifier(well)
clf.fit()
clf.accuracy()

# Demonstrating BERT & Cayde

In [ ]:
well.createBERTEncodings()

In [ ]:
well._df['head_bert']

In [ ]:
dir(well._df['head_bert'][1758])